In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F

import os
import yaml

In [2]:
TRAIN_PATH = "../data/raw/train_data/"
TEST_PATH = "../data/raw/test_data/"

In [3]:
spark = (
    SparkSession.builder.master("local[*]").getOrCreate()
)

23/07/29 11:33:04 WARN Utils: Your hostname, sergey-G3-3579 resolves to a loopback address: 127.0.1.1; using 10.4.41.7 instead (on interface wlo1)
23/07/29 11:33:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/29 11:33:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [39]:
df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .load(TRAIN_PATH + "train_data_0.pq")
)

In [40]:
ohe_columns = df.columns
ohe_columns.remove("id")
ohe_columns.remove("rn")

In [43]:
def get_columns(df, ohe_columns) -> set:
    encoded_columns = set()

    for col in ohe_columns:
        pivot_table = (
            df.groupBy('id')
            .pivot(col)
            .agg(F.count(col))
            .fillna(0)
        )

        pivot_columns = pivot_table.columns
        pivot_columns.remove("id")

        pivot_columns = list(map(lambda x: col + '_' + x, pivot_columns))

        encoded_columns = encoded_columns.union(set(pivot_columns))
    
    return encoded_columns

In [44]:
def get_columns_from_file(spark, file_path: str) -> set:
    df = (
        spark
        .read
        .format("parquet")
        .option("header", "true")
        .load(TRAIN_PATH + file_path)
    )
    
    ohe_columns = df.columns
    ohe_columns.remove("id")
    ohe_columns.remove("rn")
    
    return get_columns(df, ohe_columns)

In [47]:
all_columns = {'id', 'rn'}

for file_path in os.listdir(TRAIN_PATH):
    all_columns = all_columns.union(get_columns_from_file(spark, file_path))

In [48]:
len(all_columns)

421

In [58]:
with open("../data/transformed_data/columns.yaml", 'w') as file:
    yaml.dump(list(all_columns), file)

In [3]:
import pandas as pd

In [4]:
data = pd.read_parquet(TRAIN_PATH + "train_data_0.pq")

In [5]:
data = data[data.id < 125000]

In [6]:
data.shape

(976676, 61)

In [6]:
data = data.astype("category")
data["id"] = data.id.astype(int)
data["rn"] = data.rn.astype(int)

In [7]:
encoded_data = pd.get_dummies(data, drop_first=True)

In [8]:
encoded_data.shape

(976676, 336)

In [9]:
with open("../data/transformed_data/columns.yaml", 'r') as file:
    columns = yaml.load(file, yaml.FullLoader)

In [17]:
len(columns)

421

In [16]:
len(list(encoded_data.columns))

336

In [13]:
empty_columns = [x for x in columns if x not in encoded_data.columns]

In [15]:
len(empty_columns)

85

In [18]:
for col in empty_columns:
    encoded_data[col] = 0

/tmp/ipykernel_52098/3002524520.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_data[col] = 0
/tmp/ipykernel_52098/3002524520.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_data[col] = 0
/tmp/ipykernel_52098/3002524520.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encod

limit_output extension: Maximum message size of 10000 exceeded with 14916 characters

In [19]:
encoded_data.shape

(976676, 421)

In [20]:
columns.remove("id")
columns.remove("rn")

In [21]:
aggregated_data = encoded_data.groupby("id")[columns].sum()

In [26]:
aggregated_data[:5]

,id,pre_since_confirmed_12,pre_loans3060_8,pre_loans_credit_limit_17,pre_since_confirmed_1,enc_loans_account_holder_type_6,pre_loans5_6,is_zero_over2limit_0,pre_util_19,enc_paym_9_1,...,pre_loans530_11,pre_maxover2limit_1,pclose_flag_1,enc_paym_0_0,enc_paym_22_1,is_zero_loans6090_1,enc_paym_8_1,pre_till_fclose_13,pre_over2limit_1,rn
0,0,1,0,0,1,0,10,0,0,0,...,0,0,1,0,0,10,0,0,0,10
1,1,0,0,0,0,0,14,0,0,0,...,0,0,1,0,1,12,0,2,0,14
2,2,0,0,0,0,0,3,0,0,0,...,0,0,2,0,1,2,0,0,0,3
3,3,0,0,1,1,0,15,0,0,0,...,0,0,5,0,0,15,0,0,0,15
4,4,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [23]:
aggregated_data['rn'] = encoded_data.groupby("id")['rn'].last()

In [25]:
aggregated_data = aggregated_data.reset_index(drop=False)

In [27]:
aggregated_data.to_parquet("../data/transformed_data/train_data_0.pq")

In [30]:
spark.stop()

NameError: name 'spark' is not defined